# PyTorch 기초 학습 (Week 1)

## 학습 목표
- PyTorch 설치 상태와 MPS(GPU) 사용 가능 여부를 확인한다.
- Tensor 생성/연산/변환의 기본 흐름을 이해한다.
- NumPy와의 차이, Java 배열과의 차이를 비교한다.

## Java 개발자 관점의 비유
- Tensor ≈ Java 배열 + 벡터 연산 + GPU 가속 + 자동 미분(autograd)
- NumPy array ≈ Java 배열 + 벡터 연산 (GPU/자동 미분 없음)


In [ ]:
# =============================================================================
# 1. PyTorch 설치 확인 및 MPS 사용 가능 여부
# =============================================================================
# 학습 목표: torch 버전과 MPS(GPU) 지원 여부를 확인한다
# Java 비유: 런타임 버전과 하드웨어 가속 옵션 확인과 유사

import torch

print("✅ torch 버전:", torch.__version__)
print("✅ MPS 사용 가능 여부:", torch.backends.mps.is_available())
print("✅ MPS 사용 가능(빌드 포함) 여부:", torch.backends.mps.is_built())


In [ ]:
# =============================================================================
# 2. Tensor 생성 방법
# =============================================================================
# 학습 목표: 다양한 Tensor 생성 API를 익힌다
# Java 비유: 배열을 다양한 방식으로 초기화하는 것과 유사

# 1) 직접 생성
tensor_a = torch.tensor([1, 2, 3])
print("tensor_a:", tensor_a)

# 2) 0으로 초기화
tensor_b = torch.zeros(3, 3)
print("tensor_b:\n", tensor_b)

# 3) 1로 초기화
tensor_c = torch.ones(2, 4)
print("tensor_c:\n", tensor_c)

# 4) 정규분포 랜덤 생성
tensor_d = torch.randn(2, 3)
print("tensor_d:\n", tensor_d)

# 5) 범위 생성
tensor_e = torch.arange(0, 10, 2)
print("tensor_e:", tensor_e)


In [ ]:
# =============================================================================
# 3. NumPy와 상호 변환 (메모리 공유 주의)
# =============================================================================
# 학습 목표: numpy ↔ tensor 변환과 메모리 공유 특성을 이해한다
# Java 비유: 배열 참조를 공유하면 한쪽 변경이 반영되는 상황과 유사

import numpy as np

np_array = np.array([10, 20, 30], dtype=np.float32)
torch_tensor = torch.from_numpy(np_array)
print("numpy -> tensor:", torch_tensor)

# tensor -> numpy
back_to_numpy = torch_tensor.numpy()
print("tensor -> numpy:", back_to_numpy)

# 메모리 공유 확인 (tensor 변경 시 numpy도 변경됨)
torch_tensor[0] = 999
print("변경 후 numpy:", np_array)


In [ ]:
# =============================================================================
# 4. Tensor 기본 연산
# =============================================================================
# 학습 목표: 산술 연산, 행렬 곱셈, 전치, reshape/view 차이를 이해한다
# Java 비유: 루프 기반 연산을 벡터화해 속도를 높이는 것과 유사

x = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
y = torch.tensor([[5.0, 6.0], [7.0, 8.0]])

# 기본 산술 연산
print("x + y:\n", x + y)
print("x - y:\n", x - y)
print("x * y:\n", x * y)  # 원소별 곱
print("x / y:\n", x / y)

# 행렬 곱셈 (matmul 또는 @)
print("x @ y:\n", x @ y)
print("torch.matmul(x, y):\n", torch.matmul(x, y))

# 전치
print("x.T:\n", x.T)

# reshape vs view
z = torch.arange(0, 6)
reshape_z = z.reshape(2, 3)
view_z = z.view(2, 3)
print("reshape 결과:\n", reshape_z)
print("view 결과:\n", view_z)

# 차이 설명 (주석)
# - reshape: 필요 시 복사 발생 가능 (안전)
# - view: 메모리를 공유하는 뷰 생성 (연속 메모리 필요)


In [ ]:
# =============================================================================
# 5. GPU(MPS) 사용 및 속도 비교
# =============================================================================
# 학습 목표: CPU vs MPS 성능 차이를 확인한다
# Java 비유: CPU 연산 vs GPU 가속 연산의 처리량 차이 비교

import time

# MPS 사용 가능 여부 확인
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ MPS 디바이스 사용")
else:
    device = torch.device("cpu")
    print("⚠️ MPS 사용 불가, CPU로 진행")

# CPU 텐서 생성
cpu_tensor = torch.randn(2000, 2000)

# MPS로 전송
mps_tensor = cpu_tensor.to(device)

# 연산 속도 비교 함수
def benchmark_matmul(tensor, name: str) -> None:
    start = time.time()
    result = tensor @ tensor
    # 연산 완료 동기화
    if tensor.device.type == "mps":
        torch.mps.synchronize()
    end = time.time()
    print(f"{name} 연산 시간: {end - start:.4f}초")

# CPU 연산
benchmark_matmul(cpu_tensor, "CPU")

# MPS 연산 (가능한 경우)
if device.type == "mps":
    benchmark_matmul(mps_tensor, "MPS")

# 결과 확인용 출력
print("✅ 연산 완료")


## 자동 미분(Autograd) 기초

### 학습 포인트
- `requires_grad=True`로 계산 그래프가 생성된다.
- `backward()` 호출 시 미분값(gradient)이 누적된다.
- Autograd는 **역전파(Backpropagation)**의 핵심 메커니즘이다.

### Java 개발자 관점
- Autograd는 “수식 트리”를 만들고, 그 트리를 따라 **자동 미분**을 수행하는 시스템이다.
- 수동 미분 코드를 작성할 필요 없이, **계산 그래프 기반**으로 미분값이 전파된다.


In [ ]:
# =============================================================================
# 6. requires_grad 이해 및 계산 그래프
# =============================================================================
# 학습 목표: requires_grad로 미분 가능한 텐서를 만들고 그래프를 확인한다
# Java 비유: 실행 흐름(연산)을 기록해 나중에 역방향 계산하는 것과 유사

import matplotlib.pyplot as plt

x = torch.tensor([2.0], requires_grad=True)

y = x**2 + 2 * x + 1  # y = x^2 + 2x + 1
print("y 값:", y.item())

# 계산 그래프 정보 출력
print("y.grad_fn:", y.grad_fn)

# 시각화: 함수 곡선과 x=2 지점 표시
x_vals = torch.linspace(-5, 5, steps=100)
y_vals = x_vals**2 + 2 * x_vals + 1

plt.figure(figsize=(6, 4))
plt.plot(x_vals.numpy(), y_vals.numpy(), label="y = x^2 + 2x + 1")
plt.scatter([x.item()], [y.item()], color="red", label="x=2")
plt.title("함수 그래프와 계산 지점")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 7. 간단한 함수 미분 (y = x^2 + 2x + 1)
# =============================================================================
# 학습 목표: backward()로 미분값을 얻고 수학적 계산과 비교한다
# Java 비유: 수식의 도함수를 자동으로 계산해 값을 얻는 것과 유사

# 미분 계산
x = torch.tensor([2.0], requires_grad=True)
y = x**2 + 2 * x + 1

y.backward()  # dy/dx 계산
print("x.grad (자동 미분):", x.grad.item())

# 수학적 계산: dy/dx = 2x + 2
manual_grad = 2 * x.item() + 2
print("수학적 미분값:", manual_grad)

# 시각화: 접선의 기울기 표현
x_vals = torch.linspace(-5, 5, steps=100)
y_vals = x_vals**2 + 2 * x_vals + 1

# 접선 방정식: y = f(a) + f'(a)(x - a)
a = 2.0
fa = a**2 + 2 * a + 1
slope = manual_grad

line_vals = slope * (x_vals - a) + fa

plt.figure(figsize=(6, 4))
plt.plot(x_vals.numpy(), y_vals.numpy(), label="y = x^2 + 2x + 1")
plt.plot(x_vals.numpy(), line_vals.numpy(), linestyle="--", label="접선")
plt.scatter([a], [fa], color="red")
plt.title("미분값(기울기) 시각화")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 8. 다변수 함수 미분 (편미분)
# =============================================================================
# 학습 목표: 편미분 개념과 ∂z/∂x, ∂z/∂y 계산을 이해한다
# Java 비유: 여러 입력 변수에 대한 영향도를 각각 계산하는 것과 유사

x = torch.tensor([2.0], requires_grad=True)
y = torch.tensor([3.0], requires_grad=True)

z = x**2 + y**3  # z = x^2 + y^3
z.backward()  # 스칼라이므로 바로 backward 가능

print("∂z/∂x (자동 미분):", x.grad.item())
print("∂z/∂y (자동 미분):", y.grad.item())

# 수학적 계산
manual_dx = 2 * x.item()  # dz/dx = 2x
manual_dy = 3 * (y.item() ** 2)  # dz/dy = 3y^2

print("∂z/∂x (수학적):", manual_dx)
print("∂z/∂y (수학적):", manual_dy)

# 시각화: z = x^2 + y^3 (contour)
xs = torch.linspace(-3, 3, steps=50)
ys = torch.linspace(-3, 3, steps=50)
X, Y = torch.meshgrid(xs, ys, indexing="xy")
Z = X**2 + Y**3

plt.figure(figsize=(6, 4))
contour = plt.contourf(X.numpy(), Y.numpy(), Z.numpy(), levels=30, cmap="viridis")
plt.colorbar(contour)
plt.scatter([x.item()], [y.item()], color="red", label="(x=2, y=3)")
plt.title("다변수 함수 z = x^2 + y^3 (Contour)")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 9. 체인룰 (Chain Rule) 이해
# =============================================================================
# 학습 목표: 합성 함수에서 미분이 어떻게 전파되는지 이해한다
# Java 비유: 여러 단계 계산의 영향을 거슬러 올라가 계산하는 것과 유사

x = torch.tensor([1.5], requires_grad=True)

# 합성 함수: y = (3x + 1)^2
u = 3 * x + 1
w = u**2

w.backward()
print("자동 미분 결과 dw/dx:", x.grad.item())

# 수학적 계산
# w = (3x + 1)^2 -> dw/dx = 2(3x+1) * 3
manual_grad = 2 * (3 * x.item() + 1) * 3
print("수학적 미분값:", manual_grad)

# 시각화: 합성 함수 그래프
x_vals = torch.linspace(-3, 3, steps=100)
w_vals = (3 * x_vals + 1) ** 2

plt.figure(figsize=(6, 4))
plt.plot(x_vals.numpy(), w_vals.numpy(), label="w = (3x + 1)^2")
plt.scatter([x.item()], [w.item()], color="red", label="x=1.5")
plt.title("체인룰 적용된 합성 함수")
plt.xlabel("x")
plt.ylabel("w")
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 10. 경사하강법 시뮬레이션
# =============================================================================
# 학습 목표: f(x) = (x-3)^2 최소값을 경사하강법으로 찾는다
# Java 비유: 반복문으로 최적값을 점진적으로 찾는 방식과 유사

# 목적 함수

def f(x_val: torch.Tensor) -> torch.Tensor:
    return (x_val - 3) ** 2

# 학습률 비교
learning_rates = [0.1, 0.3]
iterations = 20

plt.figure(figsize=(7, 4))

for lr in learning_rates:
    x = torch.tensor([0.0], requires_grad=True)
    history = []

    for _ in range(iterations):
        y = f(x)
        y.backward()

        # 경사하강법 업데이트
        with torch.no_grad():
            x -= lr * x.grad

        history.append((x.item(), y.item()))
        x.grad.zero_()

    # 경로 시각화
    xs_plot = torch.linspace(-1, 6, steps=100)
    ys_plot = f(xs_plot)

    plt.plot(xs_plot.numpy(), ys_plot.numpy(), color="gray", alpha=0.5)
    plt.scatter(
        [h[0] for h in history],
        [h[1] for h in history],
        label=f"lr={lr}",
    )

plt.title("경사하강법 수렴 경로")
plt.xlabel("x")
plt.ylabel("f(x)")
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


## 선형 회귀를 처음부터 구현 (핵심 학습 루프)

### 학습 포인트
- **직접 학습 루프 작성**: 예측 → 손실 → 역전파 → 업데이트의 기본 흐름
- **backward() / grad / zero_() 역할**
  - `backward()`는 미분값 계산
  - `grad`는 기울기 저장
  - `zero_()`는 기울기 초기화(누적 방지)
- **학습률(learning rate)**은 수렴 속도와 안정성에 결정적

> 이 구조가 모든 딥러닝 학습의 기본 원리입니다.


In [ ]:
# =============================================================================
# 11. 데이터 생성 (y = 2x + 3 + noise)
# =============================================================================
# 학습 목표: 선형 회귀용 가짜 데이터를 만든다
# Java 비유: 테스트 데이터셋을 생성해 로직을 검증하는 단계와 유사

import numpy as np
import matplotlib.pyplot as plt
import torch

# 재현성을 위한 시드 고정
np.random.seed(42)
torch.manual_seed(42)

# x: 0~10 범위의 100개 샘플
x_np = np.linspace(0, 10, 100)
noise = np.random.normal(0, 1, size=x_np.shape)
y_np = 2 * x_np + 3 + noise

# Tensor 변환
x = torch.tensor(x_np, dtype=torch.float32).view(-1, 1)
y = torch.tensor(y_np, dtype=torch.float32).view(-1, 1)

# 산점도 시각화
plt.figure(figsize=(6, 4))
plt.scatter(x_np, y_np, alpha=0.7)
plt.title("선형 회귀 데이터 (y = 2x + 3 + noise)")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 12. 모델 정의 및 손실 함수 (MSE)
# =============================================================================
# 학습 목표: w, b를 학습 가능한 파라미터로 정의한다
# Java 비유: 객체의 상태(w, b)를 업데이트하며 최적화하는 방식과 유사

# 학습 가능한 파라미터 초기화
w = torch.tensor([[0.0]], requires_grad=True)
b = torch.tensor([[0.0]], requires_grad=True)

# 예측 함수
def predict(x_tensor: torch.Tensor) -> torch.Tensor:
    return w * x_tensor + b

# 손실 함수 (MSE)
def mse_loss(y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
    return ((y_pred - y_true) ** 2).mean()


In [ ]:
# =============================================================================
# 13. 경사하강법 직접 구현
# =============================================================================
# 학습 목표: backward(), grad, zero_() 역할을 직접 확인한다
# Java 비유: 반복문으로 파라미터를 갱신하며 최적값을 찾는 방식과 유사

learning_rate = 0.01
epochs = 100

loss_history = []

for epoch in range(epochs):
    # 예측
    y_pred = predict(x)

    # 손실 계산
    loss = mse_loss(y_pred, y)

    # 미분 계산
    loss.backward()

    # 파라미터 업데이트
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad

    # 그래디언트 초기화 (누적 방지)
    w.grad.zero_()
    b.grad.zero_()

    loss_history.append(loss.item())

    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}: loss={loss.item():.4f}, w={w.item():.4f}, b={b.item():.4f}")


In [ ]:
# =============================================================================
# 14. 학습 과정 시각화 및 결과 비교
# =============================================================================
# 학습 목표: 학습 곡선과 최종 회귀선을 시각적으로 확인한다
# Java 비유: 로그/그래프로 학습 상태를 모니터링하는 것과 유사

# 1) epoch별 loss 그래프
plt.figure(figsize=(6, 4))
plt.plot(loss_history, label="Loss")
plt.title("학습 손실 변화")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# 2) 최종 학습된 직선과 원본 데이터 비교
with torch.no_grad():
    y_pred_final = predict(x)

plt.figure(figsize=(6, 4))
plt.scatter(x_np, y_np, alpha=0.7, label="데이터")
plt.plot(x_np, y_pred_final.numpy(), color="red", label="학습된 직선")
plt.title("학습 결과 비교")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# 3) 실제값(2,3)과 학습값 비교
print("✅ 실제 파라미터: w=2, b=3")
print(f"✅ 학습된 파라미터: w={w.item():.4f}, b={b.item():.4f}")


## nn.Module 기반 신경망 (PyTorch 표준 구조)

### Java 개발자 비유 (Spring Boot 관점)
- `nn.Module` ≈ Java의 **abstract class** (공통 동작을 정의하고 확장)
- `__init__` ≈ **생성자** (의존성/레이어 주입)
- `forward()` ≈ **predict() 메서드** (요청 처리 로직)
- Spring Boot의 컴포넌트처럼 **레이어를 조립**해서 하나의 모델 객체로 관리

> 이 템플릿이 모든 PyTorch 모델의 기본입니다.


In [ ]:
# =============================================================================
# 15. nn.Module로 신경망 정의
# =============================================================================
# 학습 목표: nn.Module을 상속해 모델 구조를 정의한다
# Java 비유: abstract class를 상속해 구현체를 만드는 것과 유사

import torch
import torch.nn as nn


class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 레이어 정의: 10 -> 20 -> 1
        self.fc1 = nn.Linear(10, 20)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        # 순전파 정의
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


model = SimpleNet()
print(model)


In [ ]:
# =============================================================================
# 16. Loss와 Optimizer 비교 (SGD vs Adam)
# =============================================================================
# 학습 목표: 손실 함수와 최적화 알고리즘의 차이를 이해한다
# Java 비유: 전략 패턴으로 최적화 로직을 교체하는 것과 유사

# 더미 데이터 생성 (입력: 10차원, 타겟: 1차원)
torch.manual_seed(42)
inputs = torch.randn(200, 10)
true_w = torch.randn(10, 1)
true_b = torch.randn(1)
targets = inputs @ true_w + true_b + 0.1 * torch.randn(200, 1)

# 손실 함수
criterion = nn.MSELoss()

# 옵티마이저 생성 (비교용)
model_sgd = SimpleNet()
optimizer_sgd = torch.optim.SGD(model_sgd.parameters(), lr=0.01)

model_adam = SimpleNet()
optimizer_adam = torch.optim.Adam(model_adam.parameters(), lr=0.01)

print("✅ Optimizer 준비 완료: SGD, Adam")


In [ ]:
# =============================================================================
# 17. 학습 루프 템플릿
# =============================================================================
# 학습 목표: forward -> loss -> backward -> update 흐름을 이해한다
# Java 비유: 요청 처리 흐름(컨트롤러→서비스→리포지토리)을 반복하는 것과 유사

epochs = 50

# SGD 학습
loss_history_sgd = []
for epoch in range(epochs):
    # Forward pass
    outputs = model_sgd(inputs)
    loss = criterion(outputs, targets)

    # Backward pass
    optimizer_sgd.zero_grad()
    loss.backward()
    optimizer_sgd.step()

    loss_history_sgd.append(loss.item())

    # 로깅
    if epoch % 10 == 0:
        print(f"[SGD] Epoch {epoch}, Loss: {loss.item():.4f}")

# Adam 학습
loss_history_adam = []
for epoch in range(epochs):
    outputs = model_adam(inputs)
    loss = criterion(outputs, targets)

    optimizer_adam.zero_grad()
    loss.backward()
    optimizer_adam.step()

    loss_history_adam.append(loss.item())

    if epoch % 10 == 0:
        print(f"[Adam] Epoch {epoch}, Loss: {loss.item():.4f}")

# 학습 곡선 비교
plt.figure(figsize=(6, 4))
plt.plot(loss_history_sgd, label="SGD")
plt.plot(loss_history_adam, label="Adam")
plt.title("Optimizer 비교 (Loss 곡선)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 18. 모델 저장/로드
# =============================================================================
# 학습 목표: 학습된 가중치를 저장하고 복원한다
# Java 비유: 객체를 직렬화해서 저장/복원하는 것과 유사

# 모델 저장
model_path = "model.pth"
torch.save(model_adam.state_dict(), model_path)
print(f"✅ 모델 저장 완료: {model_path}")

# 모델 로드
loaded_model = SimpleNet()
loaded_model.load_state_dict(torch.load(model_path))
loaded_model.eval()  # 추론 모드
print("✅ 모델 로드 완료")


## MNIST 데이터셋 로딩 및 탐색

### 학습 포인트
- **DataLoader 역할**: 배치 처리 + 셔플링으로 학습 효율 향상
- **transform 중요성**: 정규화로 학습 안정성 확보
- **Epoch / Batch / Iteration**
  - Epoch: 전체 데이터 1회 학습
  - Batch: 한번에 처리하는 샘플 묶음
  - Iteration: 배치 1회 처리

### Java 개발자 관점
- Dataset ≈ Collection
- DataLoader ≈ Iterator + 배치 처리
- batch_size는 **메모리 사용량**과 직결


In [ ]:
# =============================================================================
# 19. MNIST 데이터셋 다운로드 및 DataLoader 생성
# =============================================================================
# 학습 목표: torchvision으로 MNIST를 로드하고 배치 단위로 처리한다
# Java 비유: Collection을 Iterator로 순회하며 배치 처리하는 방식과 유사

import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

# transform 설정 (정규화 포함)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),  # MNIST 평균/표준편차
])

# MNIST 데이터셋 로드
train_dataset = torchvision.datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform,
)

test_dataset = torchvision.datasets.MNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform,
)

# DataLoader 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=2,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
)

print("✅ MNIST 데이터셋 로드 완료")
print("- Train size:", len(train_dataset))
print("- Test size:", len(test_dataset))


In [ ]:
# =============================================================================
# 20. 데이터 배치 확인 및 샘플 시각화
# =============================================================================
# 학습 목표: 배치 크기와 이미지 형태를 확인하고 데이터 분포를 시각적으로 이해한다
# Java 비유: Iterator.next()로 배치를 가져와 구조를 확인하는 것과 유사

import matplotlib.pyplot as plt

# 배치 1개 가져오기
images, labels = next(iter(train_loader))

print("✅ 배치 크기:", images.size(0))
print("✅ 이미지 shape:", images.shape)  # (batch_size, 1, 28, 28)

# 5x5 그리드 시각화
plt.figure(figsize=(6, 6))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(images[i][0], cmap="gray")
    plt.title(str(labels[i].item()))
    plt.axis("off")
plt.suptitle("MNIST 샘플 (5x5)")
plt.tight_layout()
plt.show()

# 클래스별 샘플 1개씩 시각화
class_samples = {}
for img, label in train_dataset:
    if label not in class_samples:
        class_samples[label] = img
    if len(class_samples) == 10:
        break

plt.figure(figsize=(8, 3))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(class_samples[i][0], cmap="gray")
    plt.title(f"Class {i}")
    plt.axis("off")
plt.suptitle("클래스별 샘플 1개")
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 21. 데이터 통계 확인 (클래스 분포)
# =============================================================================
# 학습 목표: 클래스별 샘플 수를 확인해 데이터 균형을 평가한다
# Java 비유: 카운팅 집계(Count by Group)와 유사

# 클래스별 샘플 개수
class_counts = torch.bincount(train_dataset.targets)
print("✅ 클래스별 샘플 개수:")
for i, count in enumerate(class_counts.tolist()):
    print(f"Class {i}: {count}")

# 클래스 분포 시각화
plt.figure(figsize=(6, 4))
plt.bar(range(10), class_counts.numpy())
plt.title("MNIST 클래스 분포")
plt.xlabel("Class")
plt.ylabel("Count")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


## CNN 모델 학습 (MNIST)

### 실무 고려사항
- **학습 시간**: 모델/배치 크기에 따라 크게 달라짐
- **GPU 메모리**: 배치 크기를 조정해 OOM 방지
- **모니터링**: 학습/검증 지표를 함께 확인해 과적합 감지


In [ ]:
# =============================================================================
# 22. CNN 모델 정의 (입출력 크기 주석 포함)
# =============================================================================
# 학습 목표: 기본 CNN 구조를 구현하고 텐서 크기 흐름을 이해한다
# Java 비유: 레이어를 조립해 파이프라인을 구성하는 것과 유사

import torch
import torch.nn as nn


class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 입력: (N, 1, 28, 28)
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)  # -> (N, 16, 28, 28)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)  # -> (N, 16, 14, 14)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)  # -> (N, 32, 14, 14)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)  # -> (N, 32, 7, 7)

        self.flatten = nn.Flatten()  # -> (N, 32*7*7)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)  # 출력: 10 클래스

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x


cnn_model = CNN()
print(cnn_model)


In [ ]:
# =============================================================================
# 23. MPS(GPU) 설정 및 데이터 분할
# =============================================================================
# 학습 목표: MPS 사용 가능 시 GPU로 연산을 이동한다
# Java 비유: 연산 엔진을 CPU -> GPU로 교체하는 것과 유사

import time
from torch.utils.data import random_split, DataLoader

# 디바이스 설정
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ MPS 사용")
else:
    device = torch.device("cpu")
    print("⚠️ MPS 미사용, CPU로 진행")

cnn_model = cnn_model.to(device)

# Train/Validation 분할 (예: 90% / 10%)
train_size = int(len(train_dataset) * 0.9)
val_size = len(train_dataset) - train_size

train_subset, val_subset = random_split(
    train_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42),
)

train_loader_cnn = DataLoader(
    train_subset,
    batch_size=64,
    shuffle=True,
    num_workers=2,
)

val_loader_cnn = DataLoader(
    val_subset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
)

print("✅ Train/Validation 분할 완료")
print("- Train size:", len(train_subset))
print("- Val size:", len(val_subset))


In [ ]:
# =============================================================================
# 24. 학습/검증 루프 구현
# =============================================================================
# 학습 목표: 학습 과정과 검증 정확도를 확인한다
# Java 비유: 서비스 로직 수행 후 검증 테스트를 주기적으로 수행하는 것과 유사

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)

epochs = 5
train_losses = []
val_accuracies = []

start_time = time.time()

for epoch in range(epochs):
    # -------------------------
    # Training Loop
    # -------------------------
    cnn_model.train()
    running_loss = 0.0

    for inputs, targets in train_loader_cnn:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = cnn_model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader_cnn)
    train_losses.append(avg_loss)

    # -------------------------
    # Validation Loop
    # -------------------------
    cnn_model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in val_loader_cnn:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = cnn_model(inputs)
            _, predicted = torch.max(outputs, 1)

            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    val_acc = correct / total
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_loss:.4f} | Val Acc: {val_acc:.4f}")

end_time = time.time()
print(f"✅ 학습 완료 (소요 시간: {end_time - start_time:.2f}초)")

# MPS 메모리 사용량 (가능한 경우)
if device.type == "mps" and hasattr(torch.mps, "current_allocated_memory"):
    print("✅ MPS 메모리 사용량(byte):", torch.mps.current_allocated_memory())


In [ ]:
# =============================================================================
# 25. 학습 모니터링 시각화 (Loss / Val Accuracy)
# =============================================================================
# 학습 목표: 과적합 여부를 시각적으로 확인한다
# Java 비유: 로그/모니터링 대시보드로 상태를 보는 것과 유사

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
plt.title("Train Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(alpha=0.3)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label="Val Accuracy", color="orange")
plt.title("Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 26. 최종 평가 (Test Accuracy / Confusion Matrix / 클래스별 정확도)
# =============================================================================
# 학습 목표: 테스트 성능을 정량적으로 평가한다
# Java 비유: 최종 통합 테스트 결과를 정리하는 단계와 유사

from sklearn.metrics import confusion_matrix
import seaborn as sns

cnn_model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = cnn_model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

# Test Accuracy
all_preds_tensor = torch.tensor(all_preds)
all_targets_tensor = torch.tensor(all_targets)

test_acc = (all_preds_tensor == all_targets_tensor).float().mean().item()
print(f"✅ Test Accuracy: {test_acc:.4f}")

# Confusion Matrix
cm = confusion_matrix(all_targets, all_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

# 클래스별 정확도
class_correct = [0] * 10
class_total = [0] * 10

for pred, target in zip(all_preds, all_targets):
    class_total[target] += 1
    if pred == target:
        class_correct[target] += 1

print("✅ 클래스별 정확도")
for i in range(10):
    acc = class_correct[i] / class_total[i] if class_total[i] > 0 else 0
    print(f"Class {i}: {acc:.4f}")


## 상세 모델 평가 및 오류 분석

### 실무 인사이트
- 프로덕션 배포 전 **오류 패턴**과 **신뢰도**를 반드시 점검해야 한다.
- 단순 정확도 외에 **Precision/Recall/F1**을 함께 확인해 리스크를 낮춘다.

### 금융권 AI 검증 절차 비교
- 신용평가 모델과 동일하게 **오류 사례 리뷰**와 **클래스별 성능 검증**이 필수
- 규제/감사 대응을 위해 **설명 가능한 오류 분석 로그**를 남겨야 한다


In [ ]:
# =============================================================================
# 27. 예측 결과 분석 (Accuracy/Precision/Recall/F1)
# =============================================================================
# 학습 목표: 분류 모델의 핵심 지표를 종합적으로 확인한다
# Java 비유: 단순 성공률 외에 세부 지표를 함께 보는 품질 검증과 유사

import torch.nn.functional as F
from sklearn.metrics import classification_report, precision_recall_fscore_support

# 전체 테스트 셋 예측 및 확률
cnn_model.eval()
all_logits = []
all_preds = []
all_targets = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        logits = cnn_model(inputs)
        probs = F.softmax(logits, dim=1)
        _, predicted = torch.max(probs, 1)

        all_logits.append(probs.cpu())
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

all_probs = torch.cat(all_logits, dim=0)

# classification report
print("✅ Classification Report")
print(classification_report(all_targets, all_preds))

# 전체 Precision/Recall/F1 요약
precision, recall, f1, _ = precision_recall_fscore_support(
    all_targets, all_preds, average="macro"
)
print(f"✅ Macro Precision: {precision:.4f}")
print(f"✅ Macro Recall: {recall:.4f}")
print(f"✅ Macro F1-Score: {f1:.4f}")


In [ ]:
# =============================================================================
# 28. Confusion Matrix 및 혼동 패턴 분석
# =============================================================================
# 학습 목표: 어떤 숫자를 자주 헷갈리는지 확인한다
# Java 비유: 에러 케이스를 카테고리별로 집계하는 것과 유사

from sklearn.metrics import confusion_matrix
import numpy as np

cm = confusion_matrix(all_targets, all_preds)

plt.figure(figsize=(7, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix (0~9)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

# 혼동 패턴 분석: 대각선 제외 상위 오분류
cm_no_diag = cm.copy()
np.fill_diagonal(cm_no_diag, 0)

confusions = []
for i in range(10):
    for j in range(10):
        if cm_no_diag[i, j] > 0:
            confusions.append((i, j, cm_no_diag[i, j]))

confusions_sorted = sorted(confusions, key=lambda x: x[2], reverse=True)
print("✅ 자주 혼동되는 숫자 Top 5")
for actual, pred, count in confusions_sorted[:5]:
    print(f"Actual {actual} -> Pred {pred}: {count}")


In [ ]:
# =============================================================================
# 29. 오분류 사례 분석 (Top 20)
# =============================================================================
# 학습 목표: 모델이 어떤 이미지를 틀리는지 시각적으로 확인한다
# Java 비유: 실패 케이스를 샘플링해 디버깅하는 방식과 유사

# 오분류 인덱스 추출
mis_idx = [i for i, (p, t) in enumerate(zip(all_preds, all_targets)) if p != t]

# Top 20 오분류 샘플
top_n = 20
sample_idx = mis_idx[:top_n]

# 테스트 데이터에서 원본 이미지 가져오기
plt.figure(figsize=(10, 8))
for i, idx in enumerate(sample_idx):
    img, label = test_dataset[idx]
    pred = all_preds[idx]

    plt.subplot(4, 5, i + 1)
    plt.imshow(img[0], cmap="gray")
    plt.title(f"Pred:{pred} / True:{label}")
    plt.axis("off")

plt.suptitle("오분류 Top 20 사례")
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 30. 클래스별 성능 분석
# =============================================================================
# 학습 목표: 각 숫자별 정확도를 비교한다
# Java 비유: 기능별 성공률을 비교해 취약 영역을 찾는 것과 유사

class_correct = [0] * 10
class_total = [0] * 10

for pred, target in zip(all_preds, all_targets):
    class_total[target] += 1
    if pred == target:
        class_correct[target] += 1

class_acc = [
    class_correct[i] / class_total[i] if class_total[i] > 0 else 0
    for i in range(10)
]

best_class = int(np.argmax(class_acc))
worst_class = int(np.argmin(class_acc))

print("✅ 클래스별 정확도")
for i, acc in enumerate(class_acc):
    print(f"Class {i}: {acc:.4f}")

print(f"✅ 가장 잘 맞추는 클래스: {best_class}")
print(f"✅ 가장 어려운 클래스: {worst_class}")

# 막대 그래프
plt.figure(figsize=(7, 4))
plt.bar(range(10), class_acc)
plt.title("클래스별 정확도")
plt.xlabel("Class")
plt.ylabel("Accuracy")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 31. 신뢰도(Confidence) 분석
# =============================================================================
# 학습 목표: 확률 기반으로 모델 신뢰도를 점검한다
# Java 비유: 결과에 대한 확신도를 함께 확인하는 로깅과 유사

# softmax 확률 최대값 추출
max_probs, pred_classes = torch.max(all_probs, dim=1)

# 높은 확률로 틀린 케이스
high_conf_wrong = [
    i for i, (p, t, prob) in enumerate(zip(all_preds, all_targets, max_probs))
    if p != t and prob.item() >= 0.9
]

# 낮은 확률로 맞춘 케이스
low_conf_right = [
    i for i, (p, t, prob) in enumerate(zip(all_preds, all_targets, max_probs))
    if p == t and prob.item() <= 0.5
]

print("✅ 높은 확률로 틀린 케이스 수:", len(high_conf_wrong))
print("✅ 낮은 확률로 맞춘 케이스 수:", len(low_conf_right))

# 일부 사례 시각화 (최대 10개)
show_n = 10

plt.figure(figsize=(10, 4))
for i, idx in enumerate(high_conf_wrong[:show_n]):
    img, label = test_dataset[idx]
    pred = all_preds[idx]
    prob = max_probs[idx].item()

    plt.subplot(2, 5, i + 1)
    plt.imshow(img[0], cmap="gray")
    plt.title(f"P:{pred} T:{label}\nConf:{prob:.2f}")
    plt.axis("off")

plt.suptitle("높은 확률로 틀린 케이스")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
for i, idx in enumerate(low_conf_right[:show_n]):
    img, label = test_dataset[idx]
    pred = all_preds[idx]
    prob = max_probs[idx].item()

    plt.subplot(2, 5, i + 1)
    plt.imshow(img[0], cmap="gray")
    plt.title(f"P:{pred} T:{label}\nConf:{prob:.2f}")
    plt.axis("off")

plt.suptitle("낮은 확률로 맞춘 케이스")
plt.tight_layout()
plt.show()
